In [1]:
pip freeze | grep torch

pytorch-lightning==2.0.4
torch==1.12.1+cu116
torchmetrics==0.11.4
torchvision==0.13.1+cu116
Note: you may need to restart the kernel to use updated packages.


In [2]:
import argparse
import builtins

import os
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW, Adam
import torch.distributed as dist
import torch.nn as nn

import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data.distributed import DistributedSampler 
from transformers import CLIPModel, AutoTokenizer

from data.custom_data import *

In [3]:
data_root = '/data/aihub/Training/images'  
label_root = '/data/aihub/Training/labels/labels.json'

In [4]:
model_name = "EleutherAI/polyglot-ko-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)

You are using a model of type gpt_neox to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at EleutherAI/polyglot-ko-1.3b were not used when initializing CLIPModel: ['gpt_neox.layers.12.input_layernorm.bias', 'gpt_neox.layers.20.attention.dense.bias', 'gpt_neox.layers.17.post_attention_layernorm.weight', 'gpt_neox.layers.14.attention.bias', 'gpt_neox.layers.21.attention.bias', 'gpt_neox.layers.3.attention.query_key_value.bias', 'gpt_neox.layers.2.attention.query_key_value.bias', 'gpt_neox.layers.3.mlp.dense_4h_to_h.bias', 'gpt_neox.layers.9.attention.bias', 'gpt_neox.layers.6.attention.query_key_value.weight', 'gpt_neox.layers.12.attention.query_key_value.weight', 'gpt_neox.layers.11.post_attention_layernorm.weight', 'gpt_neox.embed_in.weight', 'gpt_neox.layers.2.post_attention_layernorm.weight', 'gpt_neox.layers.2.attention.dense.weight', 'gpt_neox.layers.18.attention.dense.bias', 'gpt_neox.layers.23.post_attention_layernorm.weight', 'gpt_neox.layers.18.attention.rotary_emb.inv_freq', 'gpt_neox.layers.20.mlp.dense_h_to_4h.bias', 'gpt_neo

In [5]:
model.load_state_dict(torch.load('/workspace/runs/final_model/pytorch_model.bin'))

<All keys matched successfully>

In [6]:
for p in model.parameters():
    p.requires_grad = False

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
model = model.to(device)
model.eval()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05, element

In [17]:
dataset = AIHub_data(
    root='/data/aihub/Training/images',
    json='/data/aihub/Training/labels/labels.json',
    transform=get_transforms(mode='eval'),
    tokenizer=tokenizer
    )
data_loader = DataLoader(dataset, batch_size=4, shuffle=False)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [18]:
with torch.no_grad():
    for batch, inputs in enumerate(data_loader):
        inputs = {k:v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        break

RuntimeError: Unable to find a valid cuDNN algorithm to run convolution